# NYC Incident Prediction Model

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import  RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import root_mean_squared_error

## Helper Functions

In [ ]:
def plot_timeseries(df, cols):
    fig, ax = plt.subplots(nrows=1,ncols=len(cols), figsize=(25,5))
    for i, col in enumerate(cols):
        sp = ax[i].scatter(
                df[col[0]],
                df[col[1]],
                c=df["target"]
             )
        ax[i].set_title(col[2])
    _ = fig.colorbar(sp)
    plt.show()
    
def plot_3d_timeseries(df, graphs):
    for graph in graphs:
        cols, title = graph[0], graph[1]
        scatter_plot = px.scatter_3d(df, x=cols[0], y=cols[1], z=cols[2], color='target', title=title, range_color=[0,200], height=600, width=600)
        scatter_plot.show()

## Setup

In [ ]:
df = pd.read_csv('../data/encoded_df.csv')
df.describe()

## Visualizing Time Encoded Data

Two-dimensional representation of cyclic encoding

In [ ]:
twoDim_graphs =  [['month_cos', 'month_sin','Month Of Year'], ['day_cos', 'day_sin','Day Of Month'], ['hour_cos', 'hour_sin','Hour Of Day'], ['yearday_cos', 'yearday_sin','Day Of Year'], ['weekday_cos', 'weekday_sin','Day Of Week']]
plot_timeseries(df, twoDim_graphs)

Three-dimensional representation of cyclic encoding

In [ ]:
threeDim_graphs = [
    [['day_cos', 'day_sin', 'month_of_year'], 'Target vs. Days of Month'], 
    [['hour_cos', 'hour_sin', 'day_of_year'], 'Target vs. Hourly Day of Year'], 
    [['hour_cos', 'hour_sin', 'day_of_week'], 'Target vs. Hourly Day of Week'],
    [['hour_cos', 'hour_sin', 'month_of_year'], 'Target vs. Hourly Month of Year'],
    [['month_cos', 'month_sin', 'day_of_year'], 'Target by Monthly Day of Year']
]
plot_3d_timeseries(df, threeDim_graphs)

## Model Selection

In [ ]:
# drop columns containing pre-encoded time data
original_time_cols = ['year', 'month_of_year', 'day_of_month', 'hour_of_day', 'day_of_week', 'day_of_year']
df = df.drop(original_time_cols, axis=1)

# convert columns to categorical
categorical_cols = ['is_holiday', 'is_leap_year', 'BORO_NM']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True, dtype=int)

# fix categorical conversion issue
df['BORO_NM_STATEN_ISLAND'] = df['BORO_NM_STATEN ISLAND']
df = df.drop(['BORO_NM_STATEN ISLAND'], axis=1)

# train test split
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size= 0.20)

*Basic Model Selection*:

- Linear Regression
- KNN Regression
- Random Forest Regression
- Gradient Boosting Regression

In [ ]:
clf0 = LinearRegression()
clf1 = KNeighborsRegressor(n_neighbors=5, weights='distance')
clf2 = RandomForestRegressor(n_jobs=-1, oob_score=True, random_state=1122)
clf3 = GradientBoostingRegressor(n_estimators=10)

models = [clf0, clf1, clf2, clf3]

# training data metrics
print(f"\n --- Training Data Statistics ---- \n")
print(f'Train Mean: {np.round(y_test.mean(), 2)}')
print(f'Train Median: {np.round(y_test.median(), 2)}')
print(f'Train Std Dev: {np.round(y_test.std(), 2)}')

# model metrics
for i, model in enumerate(models):
    print(f"\n --- Model {i+1} ---- \n")
    res = model.fit(X_train, y_train)
    y_pred = res.predict(X_test)
    y_pred_train = res.predict(X_train)
    print(f'RMSE Train: {np.round(root_mean_squared_error(y_pred= y_pred_train, y_true= y_train), 2)}')
    print(f'RMSE Test: {np.round(root_mean_squared_error(y_pred= y_pred, y_true= y_test), 2)}')

While Gradient Boosting appears loose to Random Forest, we can see that the random forest model is overfitting. We must also consider that gradient boosting is very sensitive to hyperparameter tuning.

I will proceed with the Gradient Boosted model as I believe that it's performance can match *(if not surpass)* the performance of the random forest model while having better control over the model overfitting.

## Gradient Boosting Model Tuning

In theory, we need to limit the depth of trees created by the gradient boosting model in order to force them into being weak learners. The core idea behind gradient boosting requires that our model iteratively creates weak learners **(stumps)** that eventually mesh together to form a strong predictor.

We will utilize GridSearchCV in order to efficiently search the parameter space.

Parameter Search Space:

- Max Depth: [1, 2, 3]
- Number of Estimators: [100, 200, 500, 1000, 2000]
- Learning Rate: [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
- Minimum Impurity Decrease: [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
- Max Features: [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [ ]:
# grid search cv
n_estimators = [10, 50, 100, 200, 500]
param_grid = [{"max_depth": [1, 2, 3], 'n_estimators': [100, 200, 500, 1000, 2000], 'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3], 'min_impurity_decrease': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3], 'max_features': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}]

gs5 = GridSearchCV(GradientBoostingRegressor(random_state=1122), param_grid=param_grid, scoring='neg_root_mean_squared_error', n_jobs=-1)
gs5.fit(X_train, y_train)

scores = gs5.cv_results_['mean_test_score']*-1
print(gs5.best_estimator_)

Results of Grid Search CV:

- Max Depth: 3
- Learning Rate: 0.475
- Minimum Impurity Decrease: 0.175
- Number of Estimators: 500
- Max Features: 1.0


## Computational Performance

*At what point do we stop creating estimators? At what point is it not worth the computation to predict 1 more incident correctly?*

We must consider the performance of our model in situations where it would be applied. Creating more estimators does not always benefit us as it affects the time complexity of the model.

In [ ]:
# plotting effect of n_estimators on Train and Test RMSE
train_scores = []
test_scores = []

n_estimators = [10, 20, 30, 50, 100, 200, 500, 1000]

for i in n_estimators:
    champion_gb = GradientBoostingRegressor(random_state=1122, learning_rate=0.475, min_impurity_decrease=0.175, n_estimators=i, max_depth=3, max_features=1.0).fit(X_train, y_train)
    cgb_res = champion_gb.predict(X_test)
    cgb_res_t = champion_gb.predict(X_train)
    train_scores.append(np.round(root_mean_squared_error(y_pred=cgb_res_t, y_true= y_train), 2))
    test_scores.append(np.round(root_mean_squared_error(y_pred=cgb_res, y_true= y_test), 2))
    
plt.plot(n_estimators, train_scores, label='Train', c='Blue')
plt.plot(n_estimators, test_scores, label='Test', c='Red')
plt.legend(loc='best')
plt.xlabel("Estimators")
plt.ylabel("RMSE")
plt.title("GB:\nTrain RMSE vs. Test RMSE")
plt.show()

## Champion Model Evaluation

In [ ]:
champion_gb = GradientBoostingRegressor(random_state=1122, learning_rate=0.475, min_impurity_decrease=0.175, n_estimators=200, max_depth=3, max_features=1.0).fit(X_train, y_train)
cgb_res = champion_gb.predict(X_test)
print(f'Champion GB RMSE: {np.round(root_mean_squared_error(y_pred=cgb_res, y_true= y_test), 2)}')

## Pack and Publish

In [ ]:
import joblib
joblib.dump(value=[champion_gb, list(X_train.columns), y_train], filename='../data/model.pkl')